In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

In [14]:
# GIS Search API

import requests

zip = 98117

# Base URL



# Default parameters
params = {
    #??Active Listings
    "al": 1,
    #Include Nearby Homes
    "include_nearby_homes": False,
    # Market. ie Seattle
    "market": None,
    # Number of homes to retrieve
    "num_homes": 350,
    #How to Sort the homes
    "ord": "days-on-redfin-asc",
    "page_number": 1,
    "poly": None,
    #Listing Types
    "sf": "1,2,3,4,5,6,7",
    "start": None,
    "status": 9,
    # User input property types
    "uipt": "1,2,3,4,5,6,7,8",
    # ??API Version?
    "v": 8,
    "zoomLevel": None,
    #Type of Region analyzed
    "region_type" : 2,
    "region_id" : get_stingray_rgn_id(zip)
    
}


url_param = build_stingray_gis_params(params)

json_data = call_stingray_gis(url_param)

list_data = parse_stingray_gis(json_data)

print(list_data)



[{'mlsId': {'label': 'MLS#', 'value': '2248199'}, 'showMlsId': False, 'mlsStatus': 'Active', 'showDatasourceLogo': True, 'price': {'value': 1200000, 'level': 1}, 'hideSalePrice': False, 'hoa': {'level': 1}, 'isHoaFrequencyKnown': True, 'sqFt': {'value': 1433, 'level': 1}, 'pricePerSqFt': {'value': 837, 'level': 1}, 'lotSize': {'value': 5100, 'level': 1}, 'beds': 3, 'baths': 2.0, 'fullBaths': 2, 'location': {'value': 'Ballard', 'level': 1}, 'stories': 2.0, 'latLong': {'value': {'latitude': 47.6773384, 'longitude': -122.385635}, 'level': 1}, 'streetLine': {'value': '6551 23rd Ave NW', 'level': 1}, 'unitNumber': {'level': 1}, 'city': 'Seattle', 'state': 'WA', 'zip': '98117', 'postalCode': {'value': '98117', 'level': 1}, 'countryCode': 'US', 'showAddressOnMap': True, 'soldDate': 1636963200000, 'searchStatus': 1, 'propertyType': 6, 'uiPropertyType': 1, 'listingType': 1, 'propertyId': 164016, 'listingId': 189124088, 'dataSourceId': 1, 'marketId': 1, 'yearBuilt': {'value': 1903, 'level': 1}, 

In [15]:
def get_stingray_rgn_id(zip):
    query_location_api = f"https://www.redfin.com/stingray/do/query-location?location={zip}&v=2"
    response = requests.get(query_location_api, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}) 
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    data = json.loads(prefix_removed)
    region_id = data["payload"]["exactMatch"].get("id").split("_",1)[1]
    return region_id


def build_stingray_gis_params(params):
        return "&".join(f"{key}={value}" for key, value in params.items() if params.get(key) != None)


def call_stingray_gis(params_url):
    api_url = "https://www.redfin.com/stingray/api/gis"
    url = f"{api_url}?{params_url}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]

    data = json.loads(prefix_removed)

    return data


def parse_stingray_gis(data):
    homes = data.get('payload', {}).get('homes', [])
    print(homes)
    parsed_homes = []
    
    for home in homes:
        home_info = {
            "MLS ID": home.get('mlsId', {}).get('value'),
            "Status": home.get('mlsStatus'),
            "Price": home.get('price', {}).get('value'),
            "HOA Fee": home.get('hoa', {}).get('value'),
            "Square Feet": home.get('sqFt', {}).get('value'),
            "Price per Square Foot": home.get('pricePerSqFt', {}).get('value'),
            "Lot Size": home.get('lotSize', {}).get('value'),
            "Bedrooms": home.get('beds'),
            "Bathrooms": home.get('baths'),
            "Location": home.get('location', {}).get('value'),
            "Stories": home.get('stories'),
            "Address": home.get('streetLine', {}).get('value'),
            "City": home.get('city'),
            "State": home.get('state'),
            "ZIP Code": home.get('postalCode', {}).get('value'),
            "Year Built": home.get('yearBuilt', {}).get('value'),
            "URL": home.get('url'),
        }
        parsed_homes.append(home_info)
    
    return parsed_homes



# print(data)


# homes = parse_json(data)

# print(homes)

# df = pd.DataFrame(homes)

# display(df)